# Cell Explorer

Below is a cell explorer written in python. It uses a couple of libraries to enable the "you" The User to:

* Load a images preferably 4. I'll send a pull request to the team responsible to figure out the bug.
* Zoom into an image, zoom out.
* Change brightness, contrast and sharpness of the image.
* Save coordinates selected in JSON (JavaScript Object Notation). 
* Write a description of the image.
* Rename the file.

This is just one part of the project. This application was made for life scientists to use to refine the features of their images to look for something of interest. 

In [1]:
# Used to change filepaths
from pathlib import Path

# We set up matplotlib, pandas, and the display function
%matplotlib inline

# graph visualization library roughly 
import matplotlib.pyplot as plt

# shows python objects i.e module of interest
from IPython.display import display

# Data manipulation library similar to dplyr 
import pandas as pd

# for arranging parts of the application
from ipywidgets import Layout

# Popular image manipulation library: Pillow
from PIL import Image, ImageEnhance

# get all modules for ipy widgets
from ipywidgets import *
import ipywidgets as widgets

# Input and output library
import io

# Linear algebra library
import numpy as np

# import Image from PIL so we can use it later
from PIL import *


In [2]:
# check the version of ipywidgets
widgets.__version__

'7.5.1'

In [3]:
# make the file upload widget
uploader = widgets.FileUpload(
    accept='.jpg',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True  # True to accept multiple files upload else False
)
display(uploader)

FileUpload(value={}, accept='.jpg', description='Upload', multiple=True)

In [4]:
def zoom_at(img, x, y, zoom):
    '''
    allows zooming in, cropping and image resizing
    
    Args:
    img an image that has to be of the form acceptable by the pillow library.
    x this is the width coordinate of the image.
    y this is the height coordinage of the image.
    zoom controls zooming in and out of an image.
    '''
    w, h = img.size
    zoom2 = zoom * 2
    img = img.crop((x - w / zoom2, y - h / zoom2, 
                    x + w / zoom2, y + h / zoom2))
    return img.resize((w, h), Image.LANCZOS)

In [6]:
@interact
def explorer(id = list(range(0, len(uploader.data))), x = 205, y = 250, zoom = 5,contrast = (0.0,5.0), brightness = (0.0,5.0), sharpness = (0.0,2.0), save_image = False):
    '''
    grabs a variable that contains images, opens one, resizes it, sets specific values for contrast, brightness,
    sharpness which is half of 5.0. If the save_image is true save the image as image-processed.jpg. Or returns an
    empty string.
    
    This is converted into a ipywidget decorated @interact
    
    Args:
    id takes the number between 0 to the length of the uploader.data method which is the number of images.
    x this is the width coordinate of the image.
    y this is the height coordinate of the image.
    zoom adjusts zoom level of the image.
    contrast adjusts the contrast of the image.
    brightness controls the light source radiating or reflecting light.
    sharpness adjusts acutance of the image.
    save_image a boolean if true saves the image and if false the image is not saved.
    
    '''
    global uploader
    ImageData = uploader.data
    ImageData = ImageData[id]
    get_image2 = Image.open(io.BytesIO(ImageData)).resize((500, 500))
    set_zoom = zoom_at(get_image2, x, y, zoom)
    img_enhance = ImageEnhance.Contrast(set_zoom).enhance(contrast)
    img_bright = ImageEnhance.Brightness(img_enhance).enhance(brightness)
    img_sharp = ImageEnhance.Sharpness(img_bright).enhance(sharpness)
    if save_image == True:
        img_sharp.save("image-processed.jpg")
    else:
        ""
    return img_sharp

interactive(children=(Dropdown(description='id', options=(0, 1, 2, 3), value=0), IntSlider(value=205, descript…

In [7]:
# make a widget that calls in a text box
writetext = widgets.Textarea(
    value='',
    placeholder='Describe the image',
    disabled=False
)

writetext

Textarea(value='', placeholder='Describe the image')

In [8]:
# defining parts of the function that user will interract with
id = explorer.widget.children[0]
x = explorer.widget.children[1]
y = explorer.widget.children[2]
zoom = explorer.widget.children[3]
contrast = explorer.widget.children[4]
brightness = explorer.widget.children[5]
sharpness = explorer.widget.children[6]
saveimage = explorer.widget.children[7]
imageviewer = explorer.widget.children[8]

In [9]:
def save_image_parameters(save_parameters=False):
    '''
    Saves the position of the image as well as the parameters specified in the left checkboxes.
    
    Args:
    save_parameters this controls the checkbox. If true the parameters of the image are saved.
    
    '''
    if save_parameters == True:
        nested_json_object = pd.DataFrame({"coordinates_x":[x.value],
                                   "coordinates_y": [y.value],
                                   "zoom": [zoom.value],
                                   "contrast":[contrast.value],
                                   "brightness":[brightness.value],
                                   "sharpness":[sharpness.value]})
        nested_json_object = str(nested_json_object.to_json(orient="records"))
        print(nested_json_object)
        file = open("saved_image_parameters.json", "w+")  
        file.write(nested_json_object)
        file.close()

In [10]:
# Makes an interractive object and saves it in a variable save_img_param
save_img_param = interact(save_image_parameters, parameters = False)

interactive(children=(Checkbox(value=False, description='save_parameters'), Output()), _dom_classes=('widget-i…

In [11]:
# grab the object with the text from the image parameters
save_img_param2 = save_img_param.widget.children[0]

In [12]:
def rename_file2(rename_file = False):
    '''
    allows user to change the name of the file that they are using in *.jpg format.
    
    Args:
    rename_file if set to True saves the file of the image one is looking using a standard format.
    '''
    if rename_file == True:
        file_ext = str(np.random.randint(100))
        change_filename = os.rename("image-processed.jpg", "img_processed" + file_ext + ".jpg")
        change_parameters = os.rename("saved_image_parameters.json", "saved_image_parameters" + file_ext + ".json")
        change_description = os.rename("saved_image_description.txt", "saved_image_description" + file_ext + ".txt")

In [13]:
# save the object containing the file paths to be renamed
rename_files = interact(rename_file2, rename_file = False)

interactive(children=(Checkbox(value=False, description='rename_file'), Output()), _dom_classes=('widget-inter…

In [14]:
# grab the object that is, the results of the renamed file
renamed_file = rename_files.widget.children[0]

In [15]:
def save_image_description(write_description = False):
    '''
    saves the description the user types in *.txt format.
    
    Args:
    write_description if set to true it will save what the user typed into the checkbox into a file in called
    saved_image_description.txt.
    '''
    if write_description == True:
        save_description = writetext.value
        file_open = open("saved_image_description.txt", "w+")
        file_open.write(save_description)
        file_open.close()

In [16]:
# make an interactive object
save_image_dsc = interact(save_image_description, parameters = False)

interactive(children=(Checkbox(value=False, description='write_description'), Output()), _dom_classes=('widget…

In [17]:
# save the object
save_img_desptn = save_image_dsc.widget.children[0]

In [18]:
# arranging the different widgets in a vertical box relating to sliders
sliders_upload = VBox([uploader,
    id,
      x, 
      y,
      zoom,
      contrast,
      brightness,
      sharpness,
      saveimage,
      ])

In [19]:
# arranging the different widgets in a vertical box relating to checkboxes
checkboxes = VBox([save_img_param2,
      save_img_desptn, renamed_file,writetext])

In [20]:
def create_expanded_button(description, button_style):
    '''
    Default function from the ipywidgets documentation to handle user interfaces.
    '''
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

top_left_button = create_expanded_button("Top left", 'info')
top_right_button = create_expanded_button("Top right", 'success')
bottom_left_button = create_expanded_button("Bottom left", 'danger')
bottom_right_button = create_expanded_button("Bottom right", 'warning')

In [21]:
# call the function and saving the results to be used later
header_button = create_expanded_button('Image explorer: For selecting the cells of interest using an explorer', 'success')
left_button = create_expanded_button('Left', 'info')
center_button = create_expanded_button('Center', "success")
right_button = create_expanded_button('Right', 'info')
footer_button = create_expanded_button('Credit: Ben Mainye', 'success')

In [22]:
# put all components of the application in one place.
my_app = AppLayout(header=header_button,
          left_sidebar=sliders_upload,
          center=imageviewer,
          right_sidebar=checkboxes,
          footer=footer_button,
          pane_widths=[10, 10, 10],
          pane_heights=[1, 10, '50px'])

display(my_app)

AppLayout(children=(Button(button_style='success', description='Image explorer: For selecting the cells of int…